## Data cleaning and wrapping

Time to download the data, clean the dataset and wrap it.

There is a .csv file for each year, but for the analysis, it's better if we work with just one data frame.

We will also need to modify some columns and add some new ones for our future analysis.

Let's start!




In [1]:
#importing the libraries that we are going to use
import pandas as pd
import numpy as np

Creating the data frame *df* with all the data. I decided to use the url directly. There is a different url for each year.

In [2]:
# links is a list with all the used url's
links=["https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/94b499d9-712a-4d2a-b790-7ceec5c9c4b1/download/311_service_requests_2011.csv",
       "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/382e10d9-1864-40ba-bef6-4eea3c75463c/download/311_service_requests_2012.csv",
       "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/407c5cd0-f764-4a41-adf8-054ff535049e/download/311_service_requests_2013.csv",
       "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/bdae89c8-d4ce-40e9-a6e1-a5203953a2e0/download/311_service_requests_2014.csv",
       "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/c9509ab4-6f6d-4b97-979a-0cf2a10c922b/download/311_service_requests_2015.csv",
       "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/b7ea6b1b-3ca4-4c5b-9713-6dc1db52379a/download/311_service_requests_2016.csv",
       "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/30022137-709d-465e-baae-ca155b51927d/download/311_service_requests_2017.csv",
       "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/311_service_requests_2018.csv",
       "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/311_service_requests_2019.csv",
       "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/script_105774672_20210108153400_combine.csv",
       "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/f53ebccd-bc61-49f9-83db-625f209c95f5/download/tmp8aj__jit.csv"]

# now we iterate over links to append each year data frame in the variable df

df=pd.DataFrame()
for link in links:
  df=df.append(pd.read_csv(link))


C:\Users\denis\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#check the head
df.head(3)

,case_enquiry_id,open_dt,target_dt,closed_dt,ontime,case_status,closure_reason,case_title,subject,reason,...,police_district,neighborhood,neighborhood_services_district,ward,precinct,location_street_name,location_zipcode,latitude,longitude,source
0,101000295613,2011-07-01 01:32:33,2011-07-15 01:32:32,2011-07-01 05:06:58,ONTIME,Closed,Case Closed Case Resolved replaced 100 amp fus...,Street Light Outages,Public Works Department,Street Lights,...,B3,Greater Mattapan,7,Ward 17,1714,INTERSECTION Groveland St & River St,NaN,42.2715,-71.0772,Employee Generated
1,101000295614,2011-07-01 02:18:31,2011-07-05 02:18:31,2011-07-01 06:12:43,ONTIME,Closed,Case Closed Case Scheduled Items have been sch...,Schedule a Bulk Item Pickup,Public Works Department,Sanitation,...,E5,Roslindale,10,Ward 20,2008,39 Lorraine St,2131.0,42.2839,-71.1447,Self Service
2,101000295615,2011-07-01 02:28:04,2011-08-15 02:28:04,2011-08-01 15:21:46,ONTIME,Closed,Case Closed Case Resolved No Sidewalk and poor...,New Sign Crosswalk or Pavement Marking,Transportation - Traffic Division,Signs & Signals,...,E5,West Roxbury,12,Ward 20,2017,INTERSECTION Corey St & Lagrange St,NaN,42.3003,-71.1698,Self Service


In [4]:
#check the tail as well to look at the most recent entries
df.tail(3)

,case_enquiry_id,open_dt,target_dt,closed_dt,ontime,case_status,closure_reason,case_title,subject,reason,...,police_district,neighborhood,neighborhood_services_district,ward,precinct,location_street_name,location_zipcode,latitude,longitude,source
224658,101004014569,2021-10-18 20:37:00,2021-10-20 08:30:00,NaN,ONTIME,Open,,Missed Leaf or Yard Trash: District 10B,Public Works Department,Sanitation,...,E13,Dorchester,13,12,1208,7 Homestead St,2121.0,42.3137,-71.0943,Constituent Call
224659,101004014612,2021-10-18 23:22:53,2021-10-20 08:30:00,NaN,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,D4,Fenway / Kenmore / Audubon Circle / Longwood,14,Ward 21,2102,8 Mountfort St,2215.0,42.3476,-71.1025,Citizens Connect App
224660,101004014620,2021-10-18 23:40:17,2021-11-02 08:30:00,NaN,ONTIME,Open,,Street Light Outages,Public Works Department,Street Lights,...,A7,East Boston,1,Ward 1,0104,58 Liverpool St,2128.0,42.3716,-71.0416,Citizens Connect App


In [5]:
#check the shape to see the size of our data
df.shape

(2143614, 29)

Our original data has more than 2 million rows/requests and 29 columns. The number of rows may vary depending on when the code ran. This happens because the 2021 data is still updated once per day.

Now it's time to add two more columns. First, we need to calculate the actual time spent to close a request by subtracting the closed date from the open date. Second, we calculate de difference between the target date and the actual closed date.


In [6]:
#convert closed_dt and open_dt from object type to datetime type and than subtract
df["elapsed_time"]=pd.to_datetime(df["closed_dt"])-pd.to_datetime(df["open_dt"])
df["elapsed_time"]

0         0 days 03:34:25
1         0 days 03:54:12
2        31 days 12:53:42
3        21 days 13:09:57
4         0 days 03:00:07
               ...       
224656                NaT
224657                NaT
224658                NaT
224659                NaT
224660                NaT
Name: elapsed_time, Length: 2143614, dtype: timedelta64[ns]

In [7]:
#convert to datetime and than subtract
df["closed_minus_target"]=pd.to_datetime(df["closed_dt"])-pd.to_datetime(df["target_dt"])
df["closed_minus_target"]


0        -14 days +03:34:26
1         -4 days +03:54:12
2        -14 days +12:53:42
3           7 days 13:09:57
4                       NaT
                ...        
224656                  NaT
224657                  NaT
224658                  NaT
224659                  NaT
224660                  NaT
Name: closed_minus_target, Length: 2143614, dtype: timedelta64[ns]

The results are in timedelta type. Note that requests that are still open get a NaT because their closed_dt is NaN.

For some analysis the timedelta type isn't ideal, so let´s convert it to a float that represents the number of days elapsed.

In [8]:
#converting timedelta to days
df["elapsed_days"]=df["elapsed_time"].dt.days
df["closed_minus_target_days"]=df["closed_minus_target"].dt.days

Ok. Now we have a data frame with the data from all the years, up to date, with the new columns added for the temporal analysis.

In [9]:
#storing df to load it in other notebooks
%store df

Stored 'df' (DataFrame)
